# Download the libraries

In [ ]:
pip install -U ipywidgets

In [2]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 689.3 kB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.7/872.7 kB 6.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import wandb
wandb.login(key="PUT YOUR API KEY HERE ")


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Upload the dataset from roboflow

In [1]:
# Here if you want to take the dataset from start and want to run the model training from the begining

# from ultralytics import YOLO


# model = YOLO('yolov8m.pt')
# model.train(data='/kaggle/input/data-yaml-khaled-annotate/data.yaml', epochs=229)



In [7]:
# upload the yaml file in kaggle from the section of Datasets in the github after that take the path of the model and put it here
model = YOLO('/kaggle/input/modeld/yolov8_motorbike_detect.pt')

In [ ]:
# model.save('/kaggle/working/yolov8_motorbike_detect.pt')

# Test the video

In [11]:
import cv2
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

# Define the boundary boxes for the lanes
left_center_box = [(300, 400), (600, 600)]  # Two-lane box (center-left)
right_lane_box = [(600, 400), (900, 600)]   # One-lane box (right lane)

# Define the counter line (adjust based on your video)
counter_line_y = 450

# Function to process the video
def process_video(video_path, output_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    
    # Define video output
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))

    motorbike_count = 0  # Initialize the counter
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Draw the static boundary boxes on the frame
        cv2.rectangle(frame, left_center_box[0], left_center_box[1], (255, 0, 0), 2)  # Left-center box (blue)
        cv2.rectangle(frame, right_lane_box[0], right_lane_box[1], (255, 0, 0), 2)   # Right-lane box (blue)

        # Draw the counter line
        cv2.line(frame, (0, counter_line_y), (frame_width, counter_line_y), (0, 0, 255), 2)  # Red counter line

        # Perform YOLOv8 detection on the frame
        results = model(frame)
        
        # Loop through detections and check motorbike positions
        for result in results[0].boxes:
            # Transfer the bounding box coordinates to the CPU
            x1, y1, x2, y2 = result.xyxy[0].cpu().numpy()  # Get bounding box coordinates
            conf = result.conf[0].cpu().item()  # Confidence score
            cls = result.cls[0].cpu().item()  # Class ID

            if conf > 0.5 and model.names[int(cls)] == "motorbike":  # Only focus on motorbikes
                motorbike_center = (int((x1 + x2) / 2), int((y1 + y2) / 2))
                
                # Check if motorbike is in the left-center or right lane box
                if left_center_box[0][0] < motorbike_center[0] < left_center_box[1][0] and left_center_box[0][1] < motorbike_center[1] < left_center_box[1][1]:
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)  # Red box
                elif right_lane_box[0][0] < motorbike_center[0] < right_lane_box[1][0] and right_lane_box[0][1] < motorbike_center[1] < right_lane_box[1][1]:
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)  # Green box
                
                # Check if motorbike crosses the counter line
                if motorbike_center[1] > counter_line_y:
                    motorbike_count += 1
                
                # Display motorbike information
                cv2.putText(frame, f'Motorbike {conf:.2f}', (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
        
        # Display the motorbike count on the frame
        cv2.putText(frame, f'Motorbike Count: {motorbike_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

        # Write processed frame to output
        out.write(frame)

    cap.release()
    out.release()

# Test the process with a sample video (replace 'input.mp4' with your actual video file)
process_video('/kaggle/input/vid1editing/Vid1Edit.mp4', '/kaggle/working/output_video.mp4')



0: 384x640 10 cars, 1 truck, 10.8ms
Speed: 2.9ms preprocess, 10.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 truck, 6.9ms
Speed: 2.7ms preprocess, 6.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 1 truck, 7.2ms
Speed: 1.9ms preprocess, 7.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 6.9ms
Speed: 2.0ms preprocess, 6.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 1 truck, 7.3ms
Speed: 2.4ms preprocess, 7.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 1 truck, 7.0ms
Speed: 2.0ms preprocess, 7.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 1 truck, 7.0ms
Speed: 2.8ms preprocess, 7.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 1 truck, 7.0ms
Speed: 2.0ms preprocess, 7.0ms inference, 1.2ms p